In [3]:
import persistqueue
import shutil
from filelock import FileLock

In [2]:
queue_path = './my_queue'

try:
    shutil.rmtree(queue_path)
    print("existing queue dir removed")
except FileNotFoundError:
    print("no existing queue dir found")

existing queue dir removed


In [3]:
q = persistqueue.SQLiteAckQueue(queue_path)

q.put("str 0")
q.put("str 1")
q.put("str 2")
q.put("str 3")
q.put("str 4")
q.put("str 5")
q.put("str 6")

print("Qeue size: ", q.size)

Qeue size:  7


In [4]:
print("size of queue before get:\t", q.size)
a = q.get()
print("item removed:\t", a)
print("size of queue after get:\t", q.size)

size of queue before get:	 7
item removed:	 str 0
size of queue after get:	 6


In [5]:
print("size of queue before ack:\t", q.size)
q.ack(a)
print("item removed:\t", a)
print("size of queue after ack:\t", q.size)

size of queue before ack:	 6
item removed:	 str 0
size of queue after ack:	 6


In [7]:
print("size of queue before nack:\t", q.size)
q.nack(a)
print("item removed:\t", a)
print("size of queue after nack:\t", q.size)

Can't find item in unack cache.


size of queue before nack:	 6
item removed:	 str 0
size of queue after nack:	 6


In [8]:
q = persistqueue.SQLiteAckQueue(queue_path)
q2 = persistqueue.SQLiteAckQueue(queue_path)

In [9]:
print(q.size)
print(q2.size)
print(q.ready_count())
print(q2.ready_count())

6
6
0
0


In [10]:
a = q.get()
print(q.size)
print(q2.size)
print(q.ready_count())
print(q2.ready_count())

5
6
0
0


In [11]:
q2 = persistqueue.SQLiteAckQueue(queue_path)
print(q.size)
print(q2.size)
print(q.ready_count())
print(q2.ready_count())

resume 1 unack tasks


5
5
1
1


In [12]:
q.ack(a)

In [13]:
print(q.size)
print(q2.size)
print(q.ready_count())
print(q2.ready_count())

5
5
0
0


In [14]:
q2 = persistqueue.SQLiteAckQueue(queue_path)

In [15]:
print(q.size)
print(q2.ready_count())

5
0


In [16]:
q3 = persistqueue.SQLiteAckQueue("./empty_queue")

In [2]:
import time
import random

In [18]:
class sleeper:
    def __init__(self, i):
        self.i = i
        
    def __call__(self): 
        random.seed(self.i)
        t = random.uniform(0, 5)
        code = "{:03d}".format(self.i)
        print("executing {}...".format(code))
        time.sleep(t)
        if t<1:
            raise Exception("Crashed")
        message = "finished {} in time {:.2f}s".format(code, t)
        print(message)
        with open('sleep_out.txt', 'w') as f:
            print(message, file=f)

In [2]:
a = sleeper(0)

NameError: name 'sleeper' is not defined

In [20]:
a()

executing 000...
finished 000 in time 4.22s


In [21]:
sleeper_list = [sleeper(i) for i in range(10)]

In [22]:
sleeper_list[6]()

executing 006...
finished 006 in time 3.97s


In [24]:
shutil.rmtree("./sleep_queue")
q4 = persistqueue.SQLiteAckQueue("./sleep_queue")
for sl in sleeper_list:
    q4.put(sl)

PicklingError: Can't pickle <class '__main__.sleeper'>: it's not the same object as __main__.sleeper

In [3]:
q = persistqueue.SQLiteAckQueue("./sleep_queue")

In [4]:

from generate_queue import sleeper

In [5]:
q.size

20

In [22]:
a = q.get()
a

In [23]:
a()

executing 008...
finished 008 in time 2.96s


In [13]:
q.size

17

In [24]:
q.nack(a)

In [25]:
q.size

20

In [ ]:
q.empty